In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
from Pytorch_utils import *

os.system('jupyter nbconvert --to python preprocessing_Pytorch.ipynb')
from preprocessing_Pytorch import *

In [ ]:
# Get list patient, treatment reponse label (label_classe), 
# diffuse/not diffuse +  present of focal lesion/not label (label_classe_diff_LFNLF), 
# diffuse/not diffuse label (label_classe_diffuse)
# present of focal lesion/not label (label_classe_isfocal)

list_patient, label_classe, label_classe_diff_LFNLF ,label_classe_diffuse, label_classe_isfocal = score_de_deauville_3()
path_label = '/home/nguyen-k/Bureau/segCassiopet/labels_seg'
list_label = os.listdir(path_label)

label_classe_1 = label_classe.copy()
data_patient = np.zeros((len(list_patient), 3))

#POSITIF/NEGATIF
for i in range(len(label_classe_diffuse)):
    if (str(label_classe_diffuse[i])=='Heterogeneous'):
        label_classe_diffuse[i]=0
    else:
        label_classe_diffuse[i]=1

In [ ]:
# get training/validation and test set for diffuse/not diffuse +  present of focal lesion/not classification
# over 5 folds
ind_list=0
test = 'Multitask_PET_L'+str(ind_list)
path_list = '/home/nguyen-k/Bureau/segCassiopet2/List_Patient_'+str(ind_list)
list_test = np.load(path_list + '/Test/list_test.npy')
test_label_classe = np.load(path_list + '/Test/test_label_classe.npy')

test_label_diffuse, test_label_dfo  = [], []

# get test set
for i in range(len(list_patient)):
        if list_patient[i] in list_test:
            test_label_diffuse.append(label_classe_diff_LFNLF[i])
            test_label_dfo.append(label_classe_diffuse[i]) # diffuse or not diffuse only
#get train and val over 5 folds
for fold in range(1,6) :
    print(f'Fold {fold}')
    path_train_val = os.path.join(path_list, 'Fold'+str(fold))
    list_train = np.load(path_train_val+'/list_train.npy')
    train_label_classe = list(np.load(path_train_val+'/train_label_classe.npy'))
    list_val = list(np.load(path_train_val+'/list_val.npy'))
    val_label_classe = list(np.load(path_train_val+'/val_label_classe.npy'))
    train_label_diffuse, train_label_dfo, val_label_diffuse, val_label_dfo  = [], [], [], []

    for i in range(len(list_patient)):
        if list_patient[i] in list_train:
            train_label_diffuse.append(label_classe_diff_LFNLF[i])
            train_label_dfo.append(label_classe_diffuse[i])
        if list_patient[i] in list_val:
            val_label_diffuse.append(label_classe_diff_LFNLF[i])
            val_label_dfo.append(label_classe_diffuse[i])

    print('len train val dfo ',len(train_label_dfo),len(val_label_dfo))
    print('len train val 4diffuse ',len(train_label_diffuse),len(val_label_diffuse))
    print('len train val label ',len(train_label_classe),len(val_label_classe))
    print('len list train',len(list_train), ' list val',len(list_val))

    print('train val dfo ', train_label_dfo.count(0),train_label_dfo.count(1),val_label_dfo.count(0),val_label_dfo.count(1))
    print('train val 4diffuse ',train_label_diffuse.count(0),train_label_diffuse.count(1),train_label_diffuse.count(2),train_label_diffuse.count(3),
                val_label_diffuse.count(0),val_label_diffuse.count(1),val_label_diffuse.count(2),val_label_diffuse.count(3))
    print('train val label ',train_label_classe.count(0),train_label_classe.count(1),train_label_classe.count(2),
                        val_label_classe.count(0),val_label_classe.count(1),val_label_classe.count(2))

    path_list_new = '/home/nguyen-k/Bureau/segCassiopet2/List_Patient_diff_'+str(ind_list)
    path_train_val_new = os.path.join(path_list_new, 'Fold'+str(fold))

    # save training/val set 
    np.save(path_train_val_new+'/list_train.npy', list_train)
    np.save(path_train_val_new+'/train_label_classe.npy', train_label_classe)
    np.save(path_train_val_new+'/train_label_classe_diffuse.npy', train_label_diffuse)
    np.save(path_train_val_new+'/train_label_classe_dfo.npy', train_label_dfo)
    np.save(path_train_val_new+'/list_val.npy', list_val)
    np.save(path_train_val_new+'/val_label_classe.npy', val_label_classe)
    np.save(path_train_val_new+'/val_label_classe_diffuse.npy', val_label_diffuse)
    np.save(path_train_val_new+'/val_label_classe_dfo.npy', val_label_dfo)

print('len list test',len(list_test))
print('test dfo ', test_label_dfo.count(0),test_label_dfo.count(1))
print('test 4diffuse ',test_label_diffuse.count(0),test_label_diffuse.count(1),test_label_diffuse.count(2),test_label_diffuse.count(3))
print('test label ',train_label_classe.count(0),train_label_classe.count(1),train_label_classe.count(2),
                        val_label_classe.count(0),val_label_classe.count(1),val_label_classe.count(2))

# save test set : list test, test label
np.save(path_list_new +'/Test/list_test.npy', list_test)
np.save(path_list_new+'/Test/test_label_classe.npy', test_label_classe)
np.save(path_list_new+'/Test/test_label_diffuse.npy', test_label_diffuse)
np.save(path_list_new+'/Test/test_label_dfo.npy', test_label_dfo)